<a href="https://colab.research.google.com/github/isabelladegen/ComputationalLogic/blob/prolexa-plus/Tests_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Testing Notebook 
Notebook that makes sure Prolexa continues to work


### Installations like demo notebook


In [1]:
!apt-get install swi-prolog -qqq > /dev/null

### Installing Prolexa
Same error as on the branch on the forked branch https://github.com/simply-logical/ComputationalLogic/

In [2]:
!yes | pip install git+https://github.com/isabelladegen/ComputationalLogic/ -qqq > /dev/null

### Instantiate Prolexa 


In [3]:
from pyswip import Prolog
import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

2021-12-16 17:55:14,982 --------------------------------------------------------------------------------
2021-12-16 17:55:14,985 The model key 'pos' now maps to 'https://huggingface.co/flair/pos-english' on the HuggingFace ModelHub
2021-12-16 17:55:14,990  - The most current version of the model is automatically downloaded from there.
2021-12-16 17:55:14,995  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt)
2021-12-16 17:55:14,998 --------------------------------------------------------------------------------
2021-12-16 17:55:15,173 loading file /root/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63


###  Some Helper functions to make the tests easier to read

In [4]:
def query(input):
  answer = meta.standardised_query(pl, input)[0]['Output']
  if isinstance(answer, str):
    return answer
  return str(answer, 'utf-8')

def test(input, expected_answer):
  print(f'Query: {input}')
  actual_answer = query(input)
  assert actual_answer == expected_answer, 'Expected Answer:  ' + expected_answer + '\n Actual answer: ' + actual_answer
  print(f'Answer: {actual_answer}\n')

def cleanup():
  print(query('forget everything'))
  meta.reset_grammar()
  meta.initialise_prolexa(pl)

### Testing existing functionality

Syntax: `test(input_query, expected_answer)`

Be careful not to add new nouns to Prolexa. The `forget all` method currently deletes all the rules about that noun but not the noun itself

In [5]:
#run before so no unexpected knowledge is haning about
cleanup()

#Tell me about... queries
print("Test 'Tell me about...' queries:\n")
test('Tell me all', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird') #extened with new rules added to prolexa
test('tell me about unknownnoun', 'I heard you say,  tell me about unknownnoun , could you rephrase that please?')
test('tell me about tweety', 'I know nothing about tweety')
test('tell me about peter', 'peter is human. peter is mortal')

#Is... queries
print("\nTest 'Is...' queries:\n")
test('Is unkownnoun human', 'I heard you say,  Is unkownnoun human , could you rephrase that please?')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Is peter mortal', 'peter is mortal')
test('Is peter human', 'peter is human')
test('Who is human', 'peter is human')
test('Who is a bird', 'opus is a bird')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')

#Explain... queries
print("\nTest 'Is...' queries:\n")
test('Explain why unknownnoun is mortal', 'I heard you say,  Explain why unknownnoun is mortal , could you rephrase that please?')
test('Explain why tweety is a bird','Sorry, I don\'t think this is the case')
test('Explain why peter is mortal', 'peter is human; every human is mortal; therefore peter is mortal')

#Adding new rules and forgeting rules
print("\nTest adding new rules:\n")
test('Peter is mortal', 'I already knew that Peter is mortal')

#add a rule and forget it again
test('Tweety is a bird', 'I will remember that Tweety is a bird')
test('Is tweety a bird', 'tweety is a bird')
test('all birds tweet', 'I will remember that all birds tweet')
test('Does tweety tweet', 'tweety tweets')
test('Explain why tweety tweets', 'tweety is a bird; every bird is a tweet; therefore tweety is a tweet') #TODO notice the really odd transaltion <a tweet>, the comandline does not do this
test('Tell me all about tweety', 'tweety is a bird. tweety is a tweet. tweety is a bird') #TODO notice the duplication in answers!!!
test('Forget that tweety is a bird', 'I erased it from my memory')
test('tell me all about tweety', 'I know nothing about tweety')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')


#run after so no unexpected knowledge is haning about for next cells
cleanup()


I am a blank slate
Test 'Tell me about...' queries:

Query: Tell me all
Answer: every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird

Query: tell me about unknownnoun
Answer: I heard you say,  tell me about unknownnoun , could you rephrase that please?

Query: tell me about tweety
Answer: I know nothing about tweety

Query: tell me about peter
Answer: peter is human. peter is mortal


Test 'Is...' queries:

Query: Is unkownnoun human
Answer: I heard you say,  Is unkownnoun human , could you rephrase that please?

Query: Is tweety a bird
Answer: Sorry, I don't think this is the case

Query: Is peter mortal
Answer: peter is mortal

Query: Is peter human
Answer: peter is human

Query: Who is human
Answer: peter is human

Query: Who is a bird
Answer: opus is a bird

Query: Does tweety fly
Answer: Sorry, I don't think this is the case


Test 'Is...' queries:

Query: Explain why unknownnoun is mortal
Answer: I he

# Testing new functionality


### Default Reasoning

In [10]:
cleanup()
#Testing new rules
test('Tell me about opus', 'opus is a bird. opus is a penguin')
test('Tell me about peep', 'peep is a bird') #This should say peep flys
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')
test('Is opus a penguin', 'opus is a penguin')
test('Is peep a penguin', 'Sorry, I don\'t think this is the case')

#Tweety is not defined as a bird so
test('Explain why tweety flies', 'Sorry, I don\'t think this is the case')
test('Explain why peep flies', 'peep is a bird; some birds are flys; therefore peep is a fly') #notice the peep is a fly -> again this does not happen on prolexa just with prolexa_plus


#Notice that 'penguins arent flys' which does work on the comandline -> prolexa plus seems to do something weird with verbs and adjectives and nouns
#test('Spill the beans', 'every human is mortal. peter is human. penguins arent flys. every bird is a fly. every penguin is a bird. opus is a penguin. peep is a bird')
cleanup()


I am a blank slate
Query: Tell me about opus
Answer: opus is a bird. opus is a penguin

Query: Tell me about peep
Answer: peep is a bird

Query: Is opus a bird
Answer: opus is a bird

Query: Is peep a bird
Answer: peep is a bird

Query: Is opus a penguin
Answer: opus is a penguin

Query: Is peep a penguin
Answer: Sorry, I don't think this is the case

Query: Explain why tweety flies
Answer: Sorry, I don't think this is the case

Query: Explain why peep flies
Answer: peep is a bird; some birds are flys; therefore peep is a fly

I am a blank slate


In [ ]:
cleanup()
print(query('tweety flies'))
print(query('tell me all'))
cleanup()
print(query('tell me all'))


###  Ideas for new funtionality
 Combine queryies test('Is peter human and mortal', 'peter is human')

 test('Is tweety a bird', 'tweety is a bird')
test('Birds fly', 'I will remember that birds fly')
test('Does tweety fly', 'tweety flies')


In [ ]:
#Interesting error about Tweety flies
# print(query('tell me all'))
# print(query('tweety flies'))
# print(query('tell me all'))

Interestingly this problem does not exist if run directly in the backend and it correctly says:
```
prolexa> "Tweety flies".
*** utterance(Tweety flies)
*** rule([(fly(tweety):-true)])
*** answer(I will remember that Tweety flies)
I will remember that Tweety flies
*** utterance(Tell me all)
*** goal(all_rules(_77782))
*** answer(every human is mortal. peter is human. tweety flies)
every human is mortal. peter is human. tweety flies
```

In [ ]:
print(query("tweety is a bird"))
print(query('tell me all'))
print(query('Tell me all about tweety'))
print(query('Does tweety fly'))
cleanup()
print(query('Does tweety fly'))
cleanup()
print(query('Explain why tweety flies'))

In [ ]:
print(query('All birds fly'))
print(query('tell me all about tweety'))
print(query('does tweety fly'))

In [ ]:
#Prove
#test('Prove Peter is human', '')